In [1]:
import os
import numpy as np
import re
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import gc

import PIL
import torch
import torchvision

import torch.utils.data as Data
from torch.utils.data.sampler import Sampler
from torch.utils.data import Dataset, DataLoader
import cv2
from PIL import Image

import torchvision.models as models
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d, Module
from torch.optim import Adam
from tqdm.notebook import tqdm as tqdm
from ipywidgets import IntProgress

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


In [2]:
class myDataset(Dataset):
    def __init__(self, x, y, transform):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        img = cv2.imread(self.x[idx], cv2.IMREAD_COLOR)
        if img is None:
            print('Not found img : ', self.x[idx])
        img = Image.fromarray(img)
        img = self.transform(img)
        return img, self.y[idx]
    def target(self, slice_arr):
        tar = []
        for idx in slice_arr:
            tar.append(self.y[idx])
        return tar

# 1.1 load and cut：讀檔並切芒果
## output : cut_img, label_index

In [64]:
import cut

# 1.1 load and cut：讀檔並切芒果
## output : cut_img, label_index

dir_path = "./../C2_TrainDev"
dest = "./../C2_TrainDev_After_Cut"

subdir = ['Dev', 'Train']
for sub in subdir:
    new_dir = os.path.join(dest, sub)
    if not os.path.isdir(new_dir):
      try:
          os.makedirs(new_dir)
      except OSError as e:
          print(e)
      else:
          print(f"Successfully created the directory {new_dir}")
path_train, box_train, label_train = cut.load_mango_csv(csv_path=f'{dir_path}/train.csv',dir_path = dir_path)
path_dev, box_dev, label_dev = cut.load_mango_csv(csv_path=f'{dir_path}/dev.csv',dir_path = dir_path)
print(path_dev[:2])
print('Start Cutting')
cut_img_train = cut.cut_mango(path_train, dest=dest, isCut=True, box=box_train)
cut_img_dev = cut.cut_mango(path_dev, dest=dest, isCut=True, box=box_dev)
print(cut_img_dev[:2])
print('Finish Cutting')

label2idx = {
    '不良-乳汁吸附': 0,
    '不良-機械傷害': 1,
    '不良-炭疽病': 2,
    '不良-著色不佳': 3,
    '不良-黑斑病': 4
}
label_index = label_train + label_dev
label = []
for check in label_index:
    new = [0,0,0,0,0]
    for check_ in check:
        new[label2idx[check_]] = 1
    label.append(new)
print(label[:10])  

path = cut_img_train + cut_img_dev
print(len(path))
print(len(label))

['./../C2_TrainDev/Dev/26519.jpg', './../C2_TrainDev/Dev/39995.jpg']
Start Cutting
Already have cut mango!
Already have cut mango!
['./../C2_TrainDev_After_Cut/Dev/26519.jpg', './../C2_TrainDev_After_Cut/Dev/39995.jpg']
Finish Cutting
[[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0]]
29449
29449


In [4]:
print(path[:10])
print(box[:10])
print(label[:10])
print(cut_img[:3])
print(len(cut_img))

['./../C2_TrainDev/Train/38414.jpg', './../C2_TrainDev/Train/03182.jpg', './../C2_TrainDev/Train/29863.jpg', './../C2_TrainDev/Train/17937.jpg', './../C2_TrainDev/Train/40878.jpg', './../C2_TrainDev/Train/29064.jpg', './../C2_TrainDev/Train/33963.jpg', './../C2_TrainDev/Train/44680.jpg', './../C2_TrainDev/Train/35482.jpg', './../C2_TrainDev/Train/04406.jpg']
[[[46.0, 146.0, 576.0, 574.0]], [[581.0, 277.0, 97.0, 93.0]], [[514.0, 538.0, 117.0, 144.0]], [[658.0, 263.0, 59.0, 74.0], [374.0, 243.0, 609.0, 334.0]], [[432.0, 160.0, 116.0, 132.0], [620.0, 289.0, 104.0, 100.0], [712.0, 161.0, 60.0, 65.0]], [[573.0, 236.0, 68.0, 46.0], [394.0, 226.0, 127.0, 96.0], [350.0, 103.0, 116.0, 52.0]], [[527.0, 307.0, 239.0, 154.0]], [[441.0, 72.0, 347.0, 547.0], [190.0, 60.0, 232.0, 197.0]], [[509.0, 443.0, 73.0, 57.0], [409.0, 509.0, 235.0, 98.0]], [[563.0, 100.0, 89.0, 158.0], [822.0, 125.0, 231.0, 570.0]]]
[['不良-機械傷害'], ['不良-機械傷害'], ['不良-機械傷害'], ['不良-機械傷害', '不良-著色不佳'], ['不良-機械傷害', '不良-機械傷害', '不良-機械傷害

In [5]:
label2idx = {
    '不良-乳汁吸附': 0,
    '不良-機械傷害': 1,
    '不良-炭疽病': 2,
    '不良-著色不佳': 3,
    '不良-黑斑病': 4
}
label_index = []
for check in label:
    new = [0,0,0,0,0]
    for check_ in check:
        new[label2idx[check_]] = 1
    label_index.append(new)
print(label_index[:10])

[[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0]]


# 1.2 不切芒果

In [40]:
def load_mango_csv(csv_path='../C2_TrainDev/train.csv'):
    label2idx = {
    '不良-乳汁吸附': 0,
    '不良-機械傷害': 1,
    '不良-炭疽病': 2,
    '不良-著色不佳': 3,
    '不良-黑斑病': 4
    }
    path = []
    box = []
    label = []
    subdir = csv_path.split('/')[-1].split('.')[0].capitalize() #[-1]意思是倒數最後一col，.capitalize()將首英文字母大寫，其他小寫
    folder = '../C2_TrainDev/'
    # subdir : Train , Dev
    # 此時我需要的輸出格式為：
    # path: 照片路徑 : ./Train/img.jpg
    # label: 標籤  : [1,0,0,0,0]
    resize_folder = f'{folder}resize/'
    if not os.path.isdir(resize_folder):
        os.makedirs(resize_folder)
    with open(csv_path, 'r', encoding='utf8') as f:        
        for line in tqdm(f):
            clean_line = re.sub(',+\n', '', line).replace('\n', '').replace('\ufeff', '').split(',')
            curr_img_path = f'{folder}{subdir}/{clean_line[0]}'
            new_img_path = f'{resize_folder}{subdir}{clean_line[0]}'
            column = 5
            curr_label = [0,0,0,0,0]
            while column <= len(clean_line):
              symptom = clean_line[column]
              if symptom in label2idx:
                curr_label[label2idx[symptom]] = 1
              column += 5
            if not os.path.isfile(curr_img_path):
                print(f'No file for path : {curr_img_path}')
                continue
            if not os.path.isfile(new_img_path):
                img = cv2.imread(curr_img_path, cv2.IMREAD_COLOR)
                img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
                cv2.imwrite(new_img_path, img)
            path.append(new_img_path)
            label.append(curr_label)
    print('data size: ')
    print(len(path), len(label))
    print(path[:6])
    print(label[:6])
    count = np.zeros(5)
    for check in label:
      count += np.array(check)
    print('不良-乳汁吸附：'+str(count[0])+' '+str(count[0]/len(label)))
    print('不良-機械傷害：'+str(count[1])+' '+str(count[1]/len(label)))
    print('不良-炭疽病：'+str(count[2])+' '+str(count[2]/len(label)))
    print('不良-著色不佳：'+str(count[3])+' '+str(count[3]/len(label)))
    print('不良-黑斑病：'+str(count[4])+' '+str(count[4]/len(label)))
    print()
    return path, label

In [41]:
path_train , label_train = load_mango_csv()
path_dev , label_dev = load_mango_csv(csv_path='../C2_TrainDev/dev.csv')

path = path_train + path_dev
label = label_train + label_dev
print(len(path))
print(len(label))
print(path[:5])
print(label[:5])


data size: 
25768 25768
['../C2_TrainDev/resize/Train38414.jpg', '../C2_TrainDev/resize/Train03182.jpg', '../C2_TrainDev/resize/Train29863.jpg', '../C2_TrainDev/resize/Train17937.jpg', '../C2_TrainDev/resize/Train40878.jpg', '../C2_TrainDev/resize/Train29064.jpg']
[[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0]]
不良-乳汁吸附：2122.0 0.08235020180068302
不良-機械傷害：419.0 0.01626047811238746
不良-炭疽病：11489.0 0.44586308599813723
不良-著色不佳：14515.0 0.5632955603849736
不良-黑斑病：953.0 0.036983855945358586




data size: 
3681 3681
['../C2_TrainDev/resize/Dev26519.jpg', '../C2_TrainDev/resize/Dev39995.jpg', '../C2_TrainDev/resize/Dev40837.jpg', '../C2_TrainDev/resize/Dev09242.jpg', '../C2_TrainDev/resize/Dev22304.jpg', '../C2_TrainDev/resize/Dev04118.jpg']
[[0, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 1, 0, 1]]
不良-乳汁吸附：308.0 0.08367291496875849
不良-機械傷害：60.0 0.016299918500407497
不良-炭疽病：1765.0 0.4794892692203206
不良-著色不佳：1938.0 0.5264873675631622
不良-黑斑病：170.0 0.04618310241782125

29449
29449
['../C2_TrainDev/resize/Train38414.jpg', '../C2_TrainDev/resize/Train03182.jpg', '../C2_TrainDev/resize/Train29863.jpg', '../C2_TrainDev/resize/Train17937.jpg', '../C2_TrainDev/resize/Train40878.jpg']
[[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0]]


# 2. binary classification and balanced：把五類分開來，變成（1,0）問題
## 使multi-label 成為 five binary-class problem
## output : 分成五類的 dataloader

In [8]:
def images_aug_balanced(cut_img , label_index ,label,transform,batchsize):
    
    data_x = []
    data_y = []
    minority_num = 0
    majority_num = 0
    for img_path , label_indices in zip(cut_img , label_index):
        if label_indices[label] == 1:
            data_y.append(1) 
            minority_num += 1
        else: 
            data_y.append(0)
            majority_num += 1
        data_x.append(img_path)
    
    dataset = myDataset(data_x, data_y, transform)
    
    #切分70%當作訓練集、30%當作驗證集
    train_size = int(0.8 * len(dataset))
    valid_size = len(dataset) - train_size
    trainset = {}
    trainset['train'],trainset['valid'] = torch.utils.data.random_split(dataset, [train_size, valid_size])
    
    minority_weight = 1/minority_num
    majority_weight = 1/majority_num
    print(f'1:{minority_weight} , 0:{majority_weight}')
    sample_weights = np.array([majority_weight, minority_weight])
    
    trainloader = {}
    labels = trainset['train'].dataset.target(trainset['train'].indices)
    labels = list(map(int, labels))
    weights = sample_weights[labels]
    print(f'train stage balanced')
    print(f'weights:{weights[:50]}')
    print(f'len:{len(weights)}\n\n')
    num_samples = len(trainset['train']) #if minority_num*4 > majority_num else int(len(trainset['train'])*(minority_num/(minority_num+majority_num))*8)
    sampler = Data.WeightedRandomSampler(weights=weights,num_samples = num_samples)
    trainloader['train'] = DataLoader(trainset['train'], batch_size = batchsize, sampler=sampler)
    
    trainloader['valid'] = DataLoader(trainset['valid'], batch_size = batchsize)
    
    return trainloader

In [9]:
# images_split = [[],[],[],[],[]]
# for image_, label_ in zip(cut_img,label_index):
#     for index,onezero in enumerate(label_):
#         if onezero == 1:
#             images_split[index].append(image_)
# for injure_place, arr in zip(label2idx.items(),images_split):
#     print(f'{injure_place}：{len(arr)}')

transform = torchvision.transforms.Compose([
torchvision.transforms.RandomHorizontalFlip(),
torchvision.transforms.RandomRotation(15, resample=PIL.Image.BILINEAR),
torchvision.transforms.ToTensor(),
torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

batchsize = 128
dataloader_0 = images_aug_balanced(cut_img , label_index, 0, transform, batchsize)
dataloader_1 = images_aug_balanced(cut_img , label_index, 1, transform, batchsize)
dataloader_2 = images_aug_balanced(cut_img , label_index, 2, transform, batchsize)
dataloader_3 = images_aug_balanced(cut_img , label_index, 3, transform, batchsize)
dataloader_4 = images_aug_balanced(cut_img , label_index, 4, transform, batchsize)

for stage in ['train' , 'valid']:
    print(f'len of 0 {stage}:{len(dataloader_0[stage])} from {len(dataloader_0[stage])*batchsize}')
    print(f'len of 1 {stage}:{len(dataloader_1[stage])} from {len(dataloader_1[stage])*batchsize}')
    print(f'len of 2 {stage}:{len(dataloader_2[stage])} from {len(dataloader_2[stage])*batchsize}')
    print(f'len of 3 {stage}:{len(dataloader_3[stage])} from {len(dataloader_3[stage])*batchsize}')
    print(f'len of 4 {stage}:{len(dataloader_4[stage])} from {len(dataloader_4[stage])*batchsize}\n')

1:0.000471253534401508 , 0:4.22904508162057e-05
train stage balanced
weights:[4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.71253534e-04 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.71253534e-04 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05 4.22904508e-05 4.22904508e-05
 4.22904508e-05 4.22904508e-05]
len:18037


1:0.002386634844868735 , 0:3.9449287940352674e-05
train stage balanced
weights:[3.94492879e-05 3.94492879e-05 3.94492879e-05 3.94492879e-05
 3.9449

In [57]:
import torch
import torch.utils.data as data
from torch.utils.data import TensorDataset as dataset
 
list(data.WeightedRandomSampler([0.1, 0.9, 0.4, 0.7, 3.0, 0.6], 5, replacement=False))

[4, 3, 1, 5, 2]

# 3. Train fine-tune model
## 使用wide resnet50_2來進行pretrained
## output : 五個ft wr

In [12]:
def train(model,name,n_epochs,train_loader,valid_loader,optimizer,criterion,batch_size,patience):
    """
    intro:
        每次epoch都 train the model , validate the model
        並計算Early Stopping
        印出 train_loss , train_acc , val_loss , val_acc
        回傳 model
    aug:
        model,n_epochs,train_loader,valid_loader,optimizer,criterion,batch_size
    output:
        model
    """
    
    best_train_loss = 100
    best_train_acc = 0
    best_val_loss = 100
    best_val_acc = 0
    best_F1 = 0
    last_epoch = 0
    history = {
        'accuracy':[],
        'val_accuracy':[],
        'loss':[],
        'val_loss':[],
        'precision':[],
        'recall':[],
        'f1':[]
    }
    
    if torch.cuda.is_available():
        model.cuda()
    else:
        print('no gpu use')
    for epoch in range(1, n_epochs+1):
        # keep track of training and validation loss
        train_loss,valid_loss = 0.0,0.0
        train_losses,valid_losses=[],[]
        train_correct,val_correct,train_total,val_total=0,0,0,0
        confusion_stacks = []

        print('running epoch: {}'.format(epoch))
        #############################################################################################################
        #                                              train the model                                              #
        #############################################################################################################
        model.train()
        for data, target in tqdm(train_loader):
            # move tensors to GPU if CUDA is available
            if torch.cuda.is_available():#train_on_gpu
                data, target = data.cuda(), target.cuda()
            else:
                print('1')
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            #calculate accuracy
            pred = output.data.max(dim = 1, keepdim = True)[1]
            train_correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            train_total += data.size(0)
            
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_losses.append(loss.item()*data.size(0))
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
        
        #############################################################################################################
        #                                            validate the model                                             #
        #############################################################################################################
        model.eval()
        for data, target in tqdm(valid_loader):
            # move tensors to GPU if CUDA is available
            if torch.cuda.is_available():#train_on_gpu
                data, target = data.cuda(), target.cuda()
                
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss =criterion(output, target)
            #calculate accuracy
            pred = output.data.max(dim = 1, keepdim = True)[1]
            val_correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            val_total += data.size(0)
            # update validation loss
            valid_losses.append(loss.item()*data.size(0))
            
            #confusion matrix
            stacked = torch.stack((target,pred.t()[0]),dim=1)
            confusion_stacks += stacked.cpu().detach().tolist()
        
        #############################################################################################################
        #                                     print train/val/cmt epoch result                                      #
        #############################################################################################################
        # calculate average losses
        train_loss=np.average(train_losses)
        valid_loss=np.average(valid_losses)
        # calculate average accuracy
        train_acc=train_correct/train_total
        valid_acc=val_correct/val_total
        print(f'\tTraining Loss: {train_loss:.3f} \tValidation Loss: {valid_loss:.3f}')
        print(f'\tTraining Accuracy: {train_acc:.3f} \tValidation Accuracy: {valid_acc:.3f}')
        
        cmt = torch.zeros(2,2, dtype=torch.int64)
        for p in confusion_stacks:
            tl, pl = p
            cmt[tl, pl] = cmt[tl, pl] + 1
        print(f'cmt\tpred:0\tpred:1\nlabel:0\t{cmt[0,0]}\t{cmt[0,1]}\nlabel:1\t{cmt[1,0]}\t{cmt[1,1]}\n')
        
        TP = cmt[1,1]
        FP = cmt[1,0]
        FN = cmt[0,1]
        TN = cmt[1,0]
        
        p = TP.double() / (TP.double() + FP.double())
        r = TP.double() / (TP.double() + FN.double())
        F1 = 2 * r * p / (r + p)
        
        print(f'precision = {p}\trecall = {r}\tF1 = {F1}\n\n')
        
        history['accuracy'].append(train_acc)
        history['val_accuracy'].append(valid_acc)
        history['loss'].append(train_loss)
        history['val_loss'].append(valid_loss)
        history['precision'].append(p)
        history['recall'].append(r)
        history['f1'].append(F1)
        
        #############################################################################################################
        #                                              Early Stopping                                               #
        #############################################################################################################
        if valid_loss > best_val_loss:
            trigger_times += 1
            print(f'trigger times: {trigger_times}\n')
            if trigger_times > patience:
                print(f'Early stopping at trigger times: {trigger_times}')
                print(f'\tLeast Training Loss: {best_train_loss:.4f} \nLeast Validation Loss: {best_val_loss:.4f}')
                print(f'\tBest Training Accuracy: {best_train_acc:.4f} \nBest Validation Accuracy: {best_val_acc:.4f}')
                last_epoch = epoch
                break
        else:
            trigger_times = 0
            torch.save(model, f'./{name}.pt')
            best_train_loss = train_loss
            best_train_acc = train_acc
            best_val_loss = valid_loss
            best_val_acc = valid_acc
    
        #############################################################################################################
        #                                                Draw picture                                               #
        #############################################################################################################
        
    x = np.arange(1,last_epoch,1)
    train_acc = history['accuracy']
    val_acc = history['val_accuracy']
    train_loss = history['loss']
    val_loss = history['val_loss']
    f1 = history['F1']
    
    fig = plt.figure(figsize=(12,4))
    fig.subplots_adjust(hspace=0.4, wspace=0.3)

    plt.subplot(2,2,1)
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.plot(x, train_acc, label='training')
    plt.plot(x, val_acc, label='validation')
    plt.legend(loc='lower right')

    plt.subplot(2,2,2)
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.plot(x, train_loss, label='training')
    plt.plot(x, val_loss, label='validation')
    plt.legend(loc='upper right')
    
    plt.subplot(2,2,3)
    plt.title("F1-score")
    plt.xlabel("Epoch")
    plt.ylabel("f1-score")
    plt.plot(x, f1, label='f1')
    plt.legend(loc='upper right')
    
    model = torch.load(f'./{name}.pt')    
    return model

# Class 0 ：wide_resnet50_2 finetune

In [13]:
if os.path.isfile(f'./model_ft_wide_resnet50_class_0.pt'):
    print(f'use finetune model_ft_wide_resnet50_class_0')
    model_ft_wide_resnet50_class_0 = torch.load(f'./model_ft_wide_resnet50_class_0.pt')
else:
    model_ft_wide_resnet50_class_0 = models.wide_resnet50_2(pretrained=True)
    num_ftrs = model_ft_wide_resnet50_class_0.fc.in_features
    model_ft_wide_resnet50_class_0.fc = nn.Linear(num_ftrs,2)
    model_ft_wide_resnet50_class_0 = model_ft_wide_resnet50_class_0.to(device)# 放入裝置
    print(model_ft_wide_resnet50_class_0)

n_epochs = 100
batch_size = 64
optimizer = torch.optim.Adam([
    {'params':model_ft_wide_resnet50_class_0.parameters()}
], lr=0.0001)
criterion = nn.CrossEntropyLoss()
patience = 3

model_ft_wide_resnet50_class_0=train(model_ft_wide_resnet50_class_0,'model_ft_wide_resnet50_class_0',n_epochs,dataloader_0['train'],dataloader_0['valid'],optimizer,criterion, batch_size , patience)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 10.76 GiB total capacity; 6.35 GiB already allocated; 198.25 MiB free; 6.86 GiB reserved in total by PyTorch)

# Class 1 ：wide_resnet50_2 finetune

In [ ]:
if os.path.isfile(f'./model_ft_wide_resnet50_class_1.pt'):
    print(f'use finetune model_ft_wide_resnet50_class_1')
    model_ft_wide_resnet50_class_1 = torch.load(f'./model_ft_wide_resnet50_class_1.pt')
else:
    model_ft_wide_resnet50_class_1 = models.wide_resnet50_2(pretrained=True)
    num_ftrs = model_ft_wide_resnet50_class_1.fc.in_features
    model_ft_wide_resnet50_class_1.fc = nn.Linear(num_ftrs,2)
    model_ft_wide_resnet50_class_1 = model_ft_wide_resnet50_class_1.to(device)# 放入裝置
    print(model_ft_wide_resnet50_class_1)

n_epochs = 100
batch_size = 16
optimizer = torch.optim.Adam([
    {'params':model_ft_wide_resnet50_class_1.parameters()}
], lr=0.0001)
criterion = nn.CrossEntropyLoss()
patience = 3
model_ft_wide_resnet50_class_1=train(model_ft_wide_resnet50_class_1,'model_ft_wide_resnet50_class_1',n_epochs,dataloader_0['train'],dataloader_0['valid'],optimizer,criterion, batch_size , patience)

In [23]:
cmt = torch.zeros(2,2, dtype=torch.int64)
TP = cmt[0,0]+1
print(TP)
print(TP.double())
print(TP.item())
x = TP.item()/2
print(x)

tensor(1)
tensor(1., dtype=torch.float64)
1
0.5


In [77]:
import torch
target = torch.Tensor([1,1,1])
print(target.shape)
target = target.unsqueeze(1)
print(target.shape)
target = target.squeeze(1)
print(target.shape)

torch.Size([3])
torch.Size([3, 1])
torch.Size([3])


In [93]:
output = [[0.51] , [0.39] , [0.] , [0.93]]
target = [[1],[0],[1],[0]]

output = torch.Tensor(output)
target = torch.Tensor(target)

y_pred_tag = torch.round(torch.sigmoid(output))
print(y_pred_tag)
correct_results_sum = (y_pred_tag == target).sum().int().item()
print(correct_results_sum)

tensor([[1.],
        [1.],
        [0.],
        [1.]])
1


In [134]:
print(target.t())
print(target.dtype)
print(target.tolist())

tensor([[1., 0., 1., 0.]])
torch.float32
[[1.0], [0.0], [1.0], [0.0]]


In [2]:
model_ft = models.wide_resnet50_2(pretrained=True)
print(model_ft)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [3]:
print(model_ft.layer3)

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Co

In [143]:
from torch.nn import Flatten
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model_ft = models.wide_resnet50_2(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,1)
model_ft=model_ft.to(device)# 放入裝置
#model_ft = model_ft.float()
#model = model.double()

# mse_full = nn.MSELoss(reduction="none")
mse = nn.MSELoss()
batch_datas = torch.randn(2, 3, 66, 220)
batch_labels = torch.randn(2)
print(type(batch_datas))
print(type(batch_labels))
                
# # predict speed
# pred = model(batch_datas)       
# print(pred)
# loss_each = mse_full( pred.flatten(), batch_labels )
# print(loss_each)
# loss_all = torch.mean(loss_each)
# print(loss_all)
# loss_all.backward()
# print(loss_all)

print(pred.flatten().dtype)
print(batch_labels.dtype)
batch_labels = batch_labels.double()
print(batch_labels.dtype)

pred = model(batch_datas)   
loss = mse(pred.flatten(), batch_labels)

# print(loss)
loss.backward()



<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.float32
torch.float32
torch.float64
torch.FloatTensor


RuntimeError: expected dtype Double but got dtype Float (validate_dtype at /pytorch/aten/src/ATen/native/TensorIterator.cpp:143)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x46 (0x7f722a8cb536 in /usr/local/lib/python3.6/dist-packages/torch/lib/libc10.so)
frame #1: at::TensorIterator::compute_types() + 0xce3 (0x7f7267feb183 in /usr/local/lib/python3.6/dist-packages/torch/lib/libtorch_cpu.so)
frame #2: at::TensorIterator::build() + 0x44 (0x7f7267fedb64 in /usr/local/lib/python3.6/dist-packages/torch/lib/libtorch_cpu.so)
frame #3: at::native::mse_loss_backward_out(at::Tensor&, at::Tensor const&, at::Tensor const&, at::Tensor const&, long) + 0x193 (0x7f7267e3bb93 in /usr/local/lib/python3.6/dist-packages/torch/lib/libtorch_cpu.so)
frame #4: <unknown function> + 0x10b7db7 (0x7f7268267db7 in /usr/local/lib/python3.6/dist-packages/torch/lib/libtorch_cpu.so)
frame #5: at::native::mse_loss_backward(at::Tensor const&, at::Tensor const&, at::Tensor const&, long) + 0x1